In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rxr
import codebase

In [2]:
datadir = '/global/scratch/users/ann_scheliga/aux_dam_datasets/GRDC_CRB/'
basin = 'colorado'

In [3]:
res_shp = codebase.load_data.load_GRanD()

In [4]:
basin_res = res_shp.loc[res_shp['RIVER'] =='Colorado River']
basin_res[['RES_NAME','DAM_NAME','RIVER','SUB_BASIN','ADMIN_UNIT']]

,RES_NAME,DAM_NAME,RIVER,SUB_BASIN,ADMIN_UNIT
486,Shadow Mountain,Shadow Mountain,Colorado River,NaN,Colorado
491,Lake Granby,Granby,Colorado River,NaN,Colorado
596,Lake Powell,Glen Canyon,Colorado River,NaN,Arizona
609,Lake Mead,Hoover,Colorado River,NaN,Nevada
615,Lake Mohave,Davis Bor,Colorado River,NaN,Arizona
628,Lake Havasu,Parker,Colorado River,NaN,Arizona
1200,Lake J.B. Thomas,Lake J.B. Thomas Dam,Colorado River,NaN,Texas
1242,NaN,Robert Lee Dam,Colorado River,NaN,Texas
1256,NaN,Simon Freese Dam,Colorado River,NaN,Texas
1279,Lake Buchanan,Buchanan Dam,Colorado River,NaN,Texas


In [5]:
country_names = basin_res['COUNTRY'].unique()
# country_names.sort()
country_names

array(['United States'], dtype=object)

In [6]:
subregions_meta = gpd.read_file(datadir + 'subregions.geojson')
stations_meta = gpd.read_file(datadir + basin + '_stationbasins.geojson')

In [7]:
res_shp.columns

Index(['GRAND_ID', 'RES_NAME', 'DAM_NAME', 'ALT_NAME', 'RIVER', 'ALT_RIVER',
       'MAIN_BASIN', 'SUB_BASIN', 'NEAR_CITY', 'ALT_CITY', 'ADMIN_UNIT',
       'SEC_ADMIN', 'COUNTRY', 'SEC_CNTRY', 'YEAR', 'ALT_YEAR', 'REM_YEAR',
       'DAM_HGT_M', 'ALT_HGT_M', 'DAM_LEN_M', 'ALT_LEN_M', 'AREA_SKM',
       'AREA_POLY', 'AREA_REP', 'AREA_MAX', 'AREA_MIN', 'CAP_MCM', 'CAP_MAX',
       'CAP_REP', 'CAP_MIN', 'DEPTH_M', 'DIS_AVG_LS', 'DOR_PC', 'ELEV_MASL',
       'CATCH_SKM', 'CATCH_REP', 'DATA_INFO', 'USE_IRRI', 'USE_ELEC',
       'USE_SUPP', 'USE_FCON', 'USE_RECR', 'USE_NAVI', 'USE_FISH', 'USE_PCON',
       'USE_LIVE', 'USE_OTHR', 'MAIN_USE', 'LAKE_CTRL', 'MULTI_DAMS',
       'TIMELINE', 'COMMENTS', 'URL', 'QUALITY', 'EDITOR', 'LONG_DD', 'LAT_DD',
       'POLY_SRC', 'geometry'],
      dtype='object')

In [8]:
print(stations_meta.columns)
print(stations_meta.shape)

Index(['grdc_no', 'river', 'station', 'area', 'altitude', 'lat_org',
       'long_org', 'lat_pp', 'long_pp', 'dist_km', 'area_calc', 'quality',
       'type', 'comment', 'source', 'geometry'],
      dtype='object')
(41, 16)


In [9]:
stations_meta['lat_lon'] = gpd.points_from_xy(y =stations_meta['lat_org'], x =  stations_meta['long_org'])
stations_points = stations_meta.set_geometry('lat_lon', crs = "EPSG:4326")
stations_points.head(2)

,grdc_no,river,station,area,altitude,lat_org,long_org,lat_pp,long_pp,dist_km,area_calc,quality,type,comment,source,geometry,lat_lon
0,4152050.0,COLORADO RIVER,"BELOW YUMA MAIN CANAL WW AT YUMA,AZ",618715.0,31.09,32.7317,-114.6319,32.7313,-114.6312,0.1,615846.2,High,Automatic,Area difference <= 5% and distance <= 5 km,hydrosheds,"MULTIPOLYGON (((-110.90830 41.35830, -110.9042...",POINT (-114.63190 32.73170)
1,4152100.0,COLORADO RIVER,"YUMA, ARIZ.",609679.0,31.35,32.7292,-114.6208,32.7313,-114.6188,0.3,615843.6,High,Automatic,Area difference <= 5% and distance <= 5 km,hydrosheds,"MULTIPOLYGON (((-110.90830 41.35830, -110.9042...",POINT (-114.62080 32.72920)


In [17]:
# # Code runs, commented for file size
# m = stations_meta.explore(style_kwds={'color':'green','fill':False})
# basin_res.explore(m=m,style_kwds=dict(color='blue'),tooltip=['DAM_NAME','RES_NAME','RIVER','AREA_SKM'])
# stations_points.explore(m=m,style_kwds=dict(color='red'),tooltip=['grdc_no','station','river','quality'])
# # subset_bbox.explore(m=m,style_kwds={'color':'black','fill':False})
# m

In [11]:
dam_name = 'Glen Canyon'
subset_gpd = codebase.area_subsets.check_for_multiple_dams(dam_name, res_shp)

In [15]:
subset_bbox = subset_gpd.geometry.buffer(0.15)

/tmp/ipykernel_2474499/2626916005.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  subset_bbox = subset_gpd.geometry.buffer(0.15)


In [24]:
subset_bbox

1865    POLYGON ((-81.66849 34.15552, -81.66041 34.153...
dtype: geometry